In [ ]:
import pandas as pd
from transformers import AutoTokenizer

In [ ]:
#loading dataset
tweets = pd.read_csv('../data/MMHS150K/MMHS150K_with_img_text.csv')
data = tweets[['tweet_text_clean', 'binary_hate', 'split']]
data = data.rename_column("binary_hate", "labels")
data.head()

In [ ]:
#load model and tokenizer
def load_pretrained(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSequenceClassification.from_pretrained(model_name)

  if torch.cuda.is_available():
    model = model.to('cuda:0')
  return tokenizer, model

#load bert
tokenizer, model = load_pretrained('bert-base-cased')

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tweet_text_clean"], padding="max_length", truncation=True)

tokenized_data = data.map(tokenize_function, batched=True)

In [ ]:
print(tokenized_data)

In [ ]:
print(tokenized_data[["split"]=="train"][0]["tweet_text_clean"])
print(tokenized_data["train"][0]["labels"])
print(tokenized_data["train"][0]["input_ids"])
print(tokenized_data["train"][0]["attention_mask"])

In [ ]:
# Remove the text column because the model does not accept raw text as an input

tokenized_data = tokenized_data.remove_columns(["tweet_text_clean"])
print(tokenized_data)
print(type(tokenized_data), type(tokenized_data["train"]["labels"]))

In [ ]:
# Set the format of the dataset to return PyTorch tensors instead of lists


tokenized_data.set_format("torch")
print(tokenized_data)
print(type(tokenized_data), type(tokenized_data["train"]["labels"]))